## 1. Import libraries.

In [38]:
import pandas as pd
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

## 2. Load count data.

In [4]:
def make_simple_table(SRR):
    df = pd.read_csv(SRR, sep="\t", skiprows=1).iloc[:,[0,6]]
    df.columns = ["Geneid", SRR]
    return df

files = !ls example/count/*.count
dfs = [make_simple_table(i) for i in files]

## 3. Make matrix.

In [5]:
df = dfs[0]
for i in range(1,len(dfs)):
    df = pd.merge(df, dfs[i], on="Geneid")
    
df.index = df["Geneid"].map(lambda x : x.split("gene:")[1])
df.columns = ["Geneid"] + [i.split("/")[-1].split(".count")[0] for i in list(df.columns[1:])]
df = df.T.drop("Geneid")

## 4. Conver to RPKM.

In [6]:
# total reads info
trimo = !ls example/stats/trimo/*.txt
reads = {i.split("/")[-1].split(".txt")[0]:[j.split("Surviving: ")[1].split(" ")[0] for j in open(i).readlines() if "Surviving" in j][0] for i in trimo}

In [7]:
# gene length info
gff =  [i.split("\t") for i in open("example/src/Homo_sapiens.GRCh38.103.gff3").readlines() if "#" not in i and "ID=gene" in i]
gff2 = {i[8].split(";")[0].split("gene:")[1]:str(int(i[4])-int(i[3])) for i in gff}

In [34]:
# rpkm
coeff = np.array([[(10**9)/(int(reads[i])*int(gff2[j])) for j in list(df.columns)] for i in list(df.index)])
df2 = pd.DataFrame(np.array(df) * coeff)
df2.index = df.index
df2.columns = df.columns
df = df2

## 5. Add meta info.

In [35]:
control = pd.read_csv("control.txt", sep=" ", header=None).values
cocaine = pd.read_csv("cocaine.txt", sep=" ", header=None).values
y = [0 if i in control else 1 for i in list(df.index)]
df["y"] = y

## 6. Correlation coefficients.

In [ ]:
r = [np.corrcoef([list(df.iloc[:,i]), list(df.iloc[:,-1])])[0,1] for i in range(len(df.T)-1)]
r_i = [i[0] for i in sorted([i for i in enumerate(r)],key=lambda x: x[1])]
r_ = r_i[-5:] + r_i[:5]
df_ = df.iloc[:, r_+[-1]]

## 7. PCA

In [41]:
def draw_PCA(df):
    pca = PCA(n_components=2)
    result = pca.fit_transform(df.iloc[:,:-1])

    df_res = pd.DataFrame(result)
    df_res.columns = ["x", "y"]
    df_res.index = df.index
    df_res["meta"] = df["y"].tolist()
    
    df_res_0 = df_res[df_res["meta"]==0]
    df_res_1 = df_res[df_res["meta"]==1]
    
    source1 = ColumnDataSource(
        data=dict(
            x=df_res_0["x"],
            y=df_res_0["y"],
            desc=df_res_0.index.tolist()
        )
    )

    source2 = ColumnDataSource(
        data=dict(
            x=df_res_1["x"],
            y=df_res_1["y"],
            desc=df_res_1.index.tolist()
        )
    )
    
    hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("desc", "@desc"),
        ]
    )

    p = figure(tools=[hover, "save"], plot_width=550, plot_height=500)
    p.xaxis.major_label_text_color = "white"
    p.yaxis.major_label_text_color = "white"
    #p.xaxis.axis_label = 'PC1'
    #p.yaxis.axis_label = 'PC2'

    p.circle("x", "y", fill_color="black", line_color="black", fill_alpha=1, size=14, source=source1)
    p.triangle("x", "y", fill_color="red", line_color="red", fill_alpha=1, size=16, source=source2)
    
    show(p)

In [3]:
draw_PCA(df_)